# Step involved to solved the problem
 
* Downloading video from youtube
* converting videos into audio mp3 file 
* converting mp3 audio into wav audio file
* converting audio into transcrip file
* use model to solve the problem


In [ ]:
#  Downloading videos from youtube 
#------------------------------------------#
# libraries used to solve the problem
from docx import Document           # used to read the list of videos 
import youtube_dl                  # used to download the videos
#################################
# reading the list of videos
document = Document('youtube_list.docx')        # file should be in the same directory 
youtube_files= []
n = len(document.paragraphs)                   # total no of videos that will be download
# extracting the link from the document and storing in the youtube/_files
for i in range(0 ,n):
    z = document.paragraphs[i].text
    # removing all charactor that comes before https
    if i < 9:
        z = z[3:]
    else:
        z =z[4:]
    youtube_files.append(z)
# passing the list videos link and converting and downloading the  audio 
ydl_opts = {'format':'bestaudio/best' ,'postprocessors':[{'key':'FFmpegExtractAudio' ,'preferredcodec':'mp3' ,'preferredquality':'192',}]}
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    for i in range(0,n):
        ydl.download([youtube_files[i]])


In [5]:
#  converting mp3 wav file
#---------------------------------------------------#

def mp3_to_wav(file_dir):
    # libraries used
    import os
    from pydub import AudioSegment
    #-----------------------------------------#
    mp3_file = os.listdir(file_dir +str('/mp3'))
    wav_dir = file_dir + str('/wav')
    for index ,file in enumerate(mp3_files):
        name = file.split('.')[0]
        extention = file.split('.')[1]
        sound = AudioSegment.from_mp3(file)
        sound.export(wav_dir + name + '.wav' ,format ="wav") 
#-------------------------------------------------#

# converting audio into transcrip file using google speech API
* Reference:
            * https://cloud.google.com/speech-to-text/docs/quickstart-client-libraries # forsetting environment 
            * https://googleapis.github.io/google-cloud-python/latest/core/auth.html    #  authorization
            * https://cloud.google.com/storage/docs/uploading-objects#storage-upload-object-code_sample  #                     uploading audio file
            
            * https://cloud.google.com/speech-to-text/docs/async-recognize  # code for converting long audion files

In [6]:
# 
def transcribe_gcs(gcs_uri):
    """Asynchronously transcribes the audio file specified by the gcs_uri."""
    from google.cloud import speech
    from google.cloud.speech import enums
    from google.cloud.speech import types
    client = speech.SpeechClient()

    audio = types.RecognitionAudio(uri=gcs_uri)
    config = types.RecognitionConfig(
        encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
        language_code='en-US')

    operation = client.long_running_recognize(config, audio)

    print('Waiting for operation to complete...')
    response = operation.result(timeout=3600)
    text1 =[]

    # Each result is for a consecutive portion of the audio. Iterate through
    # them to get the transcripts for the entire audio file.
    for result in response.results:
        # The first alternative is the most likely one for this portion.
        #print(u'Transcript: {}'.format(result.alternatives[0].transcript))
        for alternative in result.alternatives:
            text1.append(alternative.transcript)   # storing only text 
        #print('Confidence: {}'.format(result.alternatives[0].confidence))
    return text1

In [7]:
def savetxt():
    fix_dir = "gs://vajir1bucket/wav/wav/"         # starting name of  directory of google storage bucket
    wav_files = os.listdir('/home/vajir/Desktop/gcloud/wav/') # name of the wav file 
    txt_files = '/home/vajir/Desktop/gcloud/text/' # all the converted text file will be store in this folder
    for file in wav_files:
        gcs_uri =  fix_dir + file
        text = transcribe_gcs(gcs_uri) # it will return list of text in a videos
        name = file.split('.')[0]
        # combining all text from the list and saving  transcript file into text directory
        with open(txt_files+ name +str('.txt') ,'w') as f:
            for txt in text:
                f.write(txt)